In [1]:
import re, json, pathlib, logging, time, argparse, pprint 
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import postprocess.postprocess as pspr


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [14]:
input_dir=pathlib.Path('output/DirD_/output/2-doctext')
files = sorted(input_dir.glob('**/*.txt'))
file_df=pd.DataFrame(files,columns=['Path'])
file_df['File']=file_df['Path'].apply(lambda x: x.name)
file_df['PRJ']=file_df['File'].str.extract(r'_PRJ_(\d{5})')
file_df['PP']=file_df['File'].str.extract(r'_PP_(\d{5})')

file_df.head()

,Path,File,PRJ,PP
0,output/DirD_/output/2-doctext/WP_2021_PP_20140...,WP_2021_PP_20140_PRJ_30690_13072021_docx.txt,30690,20140
1,output/DirD_/output/2-doctext/WP_2021_PP_20140...,WP_2021_PP_20140_PRJ_30701_13072021_docx.txt,30701,20140
2,output/DirD_/output/2-doctext/WP_2021_PP_20140...,WP_2021_PP_20140_PRJ_30743_13072021_docx.txt,30743,20140
3,output/DirD_/output/2-doctext/WP_2021_PP_20143...,WP_2021_PP_20143_PRJ_30323_13072021_docx.txt,30323,20143
4,output/DirD_/output/2-doctext/WP_2021_PP_20143...,WP_2021_PP_20143_PRJ_30325_13072021_docx.txt,30325,20143


In [22]:
doc_text={}
for file_ in files:
    with open(file_, 'r') as f:    
        doc_text[file_.name]=f.read()
    doc_text[file_.name]=re.findall(r"SDG\n\n([a-zA-z, -]{1,})\n", doc_text[file_.name], flags=re.IGNORECASE)

In [28]:
#for name, value in doc_text.items():
    #print(name, value)

file_df['SDG']=file_df['File'].apply(lambda x: doc_text[x][0])
file_df.head()

,Path,File,PRJ,PP,SDG
0,output/DirD_/output/2-doctext/WP_2021_PP_20140...,WP_2021_PP_20140_PRJ_30690_13072021_docx.txt,30690,20140,Climate Action
1,output/DirD_/output/2-doctext/WP_2021_PP_20140...,WP_2021_PP_20140_PRJ_30701_13072021_docx.txt,30701,20140,Climate Action
2,output/DirD_/output/2-doctext/WP_2021_PP_20140...,WP_2021_PP_20140_PRJ_30743_13072021_docx.txt,30743,20140,Climate Action
3,output/DirD_/output/2-doctext/WP_2021_PP_20143...,WP_2021_PP_20143_PRJ_30323_13072021_docx.txt,30323,20143,"Industry, Innovation and Infrastructure"
4,output/DirD_/output/2-doctext/WP_2021_PP_20143...,WP_2021_PP_20143_PRJ_30325_13072021_docx.txt,30325,20143,Responsible Consumption and Production


In [24]:
dird_df=pd.read_excel('input/Projects_overview.xlsx')
dird_df.head()

,PP,PP Acronym,PRJ,PRJ Acronym,Policy Area,PRJ Title,Unit,Main Pol. DG,Fundings,Start date,End date,Status,WKF Status
0,20140,LANDCLIM,30690,AFOLU GHGI,1.1 2021,GHG inventory monitoring and development in Ag...,JRC.D.1,CLIMA\n,CON-AA 35608\n INS-RS\n,01/01/2021,31/12/2022,ACTIVE,Draft
1,20140,LANDCLIM,30701,FORMIT,1.1 2021,Forest mitigation contribution to climate neut...,JRC.D.1,CLIMA\n,INS-RS\n,01/01/2021,31/12/2022,ACTIVE,Draft
2,20140,LANDCLIM,30743,INTEGRATE-AFOLU,1.1 2021,"Integrate Agriculture, Forestry and Land Use S...",JRC.D.4,CLIMA\n,INS-UR\n,01/01/2021,31/12/2022,ACTIVE,Draft
3,20143,SVC-RAWMAT,30323,RMIS,1.5 2021,Raw Materials Information System,JRC.D.3,GROW\n,CON-AA 35889\n INS-RS\n,01/01/2021,31/12/2022,ACTIVE,Draft
4,20143,SVC-RAWMAT,30325,SureVAL,1.5 2021,SUstainable and secure sourcing of Raw materia...,JRC.D.3,GROW\n,CON-AA 35889\n INS-RS\n,01/01/2021,31/12/2022,ACTIVE,Draft


In [34]:
file_df['PRJ']=file_df['PRJ'].astype(str)
dird_df['PRJ']=dird_df['PRJ'].astype(str)
final=dird_df.merge(file_df[['PRJ','File','SDG']], on='PRJ',how='left')

In [36]:
final.head()
final.to_excel('dird_project_summary.xlsx')